In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [3]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

### Preprocess 

In [4]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./dataset', train=True, download=True,
                      transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,),(0.3081,))
                      ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./dataset', train=False,
                      transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,),(0.3081,))
                      ])),
    batch_size=batch_size, shuffle=True)

100.1%

Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw


113.5%

Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw


100.4%

Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw


180.4%

Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw
Processing...
Done!


../torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


### Model 

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Optimization 

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [8]:
epochs = 1
log_interval = 100

In [9]:
for epoch in range(1, epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.239657
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.133956
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.007254
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.656327
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.400271
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.864159
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.613159
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.615114
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.427833


### Evaluation 

* model.eval()로 설정
    * BN, Dropout같은 layer 잠금

In [10]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

* autograd engine, 즉 backpropagation이나 gradient 계산을 꺼서 memory usage를 줄이고 속도를 높임

In [12]:
test_loss = 0
correct = 0

with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()

In [13]:
output.shape

torch.Size([64, 10])

In [14]:
target

tensor([6, 9, 0, 9, 5, 7, 6, 3, 6, 7, 8, 0, 1, 1, 9, 4, 5, 1, 7, 1, 7, 4, 2, 0,
        5, 6, 1, 4, 1, 6, 6, 3, 9, 8, 5, 5, 2, 8, 0, 0, 7, 9, 8, 9, 9, 5, 2, 2,
        5, 8, 7, 8, 1, 5, 7, 9, 5, 9, 7, 8, 7, 9, 8, 5])

In [18]:
target.view_as(pred).shape

torch.Size([64, 1])

In [21]:
pred.eq(target.view_as(pred)).sum().item()

49

In [23]:
pred.shape

torch.Size([64, 1])

In [24]:
test_loss

43.5202522277832

In [25]:
correct

49

In [26]:
test_loss /= len(test_loader.dataset)
test_loss

0.00435202522277832

#### All

In [29]:
for epoch in range(1, epochs + 1):
    # Train
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))
    
    
    # Test
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), correct/len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.141542
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.171971
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.197680
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.202623
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.206045
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.185290
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.145879
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.347655
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.085267
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.203485

Test set: Average Loss: 0.1820, Accuracy: 9476/10000 (1%)

